In [1]:
import os
import math
import numpy as np

In [45]:
# replace the join function's argument with your output file name
reactant_file = os.path.join('conformerTwoFreq.log') 
with open(reactant_file, "r") as reactantfile:
    reactant = reactantfile.readlines()

# Replace with the amount of atoms in your molecule
atom_number = 16 
hessian_size = atom_number*3

In [46]:
hessian_start_line = 0
hessian_end_line = 0


# Searches for when the section of the output file where the Hessian is
for line_number, line in enumerate(reactant, start = 1):
    if 'Hessian after L703:' in line:
        hessian_start_line = line_number


# Determine the end of the Hessian section in the output
hessian_size_counter = hessian_size
hessian_section_length = 0
hessian_column_counter = 0
while(hessian_size_counter >= 0):
    hessian_section_length += hessian_size_counter
    hessian_size_counter -=5
hessian_end_line = hessian_start_line + hessian_section_length + math.ceil(hessian_size/5)


# Identify all the line breaks, and create a hessian 
hessian = []
line_breaks = []
for line_number, line in enumerate(reactant, start = 1):

    if line_number > hessian_start_line + 1:
        current_line = line.split()

        # This condition will determine the line number of all the lines that just have the column indices
        if (all(len(s) < 4 for s in current_line)):
            line_breaks.append(line_number)

    if line_number == hessian_end_line:
        break


# Continue through for the rest of the hessian section of the output file. 
for line_number, line in enumerate(reactant, start = 1):
    if line_number > hessian_start_line + 1:
        current_line = line.split()

        # Start the matrix by first establishing the rows we will have, which will be the amount of lines before the first line break.
        if line_number < line_breaks[0]:
            hessian.append(current_line)

        # Continue going through the Hessian section of the output file, and don't count the line breaks
        if line_number > line_breaks[0] and line_number not in line_breaks:
            for element in hessian:
                # If the indexes of the rows match, then append. 
                if current_line[0] == element[0]:
                    for item in current_line:
                        # This just works lol #idk
                        if len(item) > 3:
                            hessian[int(element[0])-1].append(item)
    # Stop the loop when we reach the end of the Hessian section
    if line_number == hessian_end_line:
        break


# Remove all the row indexes, which are the first element of the rows
for item in hessian:
    item.pop(0)
